In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [ ]:
# school
# hostip = "10.192.40.238"
# home
# hostip = "192.168.0.88"
hostip = "192.168.0.160"

topic = 'ProducerA'
file_path = 'FIT3182-A2/data/camera_event_A.csv'


def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [ ]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": int(event['camera_id']),
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "A"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": int(event['camera_id']),
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "A"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')